In [1]:
%run utils.ipynb

import requests

In [3]:
response = requests.get("http://iceberg.io/v1/namespaces")
response.json()

{'namespaces': [['ecommerce']]}

In [11]:
spark = get_spark(catalog="iceberg", storage="storage.io")
object_name = f"s3a://ecommerce/raw/customers.csv"
df = spark.read.csv(object_name)
df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------+
|                 _c0|                 _c1|                 _c2|                 _c3|           _c4|
+--------------------+--------------------+--------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code...|       customer_city|customer_state|
|06b8999e2fba1a1fb...|861eff4711a542e4b...|               14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|               09790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|               01151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|               08775|     mogi das cruzes|            SP|
+--------------------+--------------------+--------------------+--------------------+--------------+
only showing top 5 rows



In [12]:
df.writeTo("iceberg.ecommerce.customers") \
    .option("write.parquet.compression-codec", "snappy") \
    .createOrReplace()

In [13]:
response = requests.get("http://iceberg.io/v1/namespaces")
response.json()

{'namespaces': [['ecommerce']]}

In [7]:
response = requests.get("http://iceberg.io/v1/namespaces/ecommerce/tables")
response.json()

{'identifiers': [{'namespace': ['ecommerce'], 'name': 'customers'}]}

In [14]:
spark.sql("show tables from iceberg.ecommerce").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|ecommerce|customers|      false|
+---------+---------+-----------+



In [16]:
response = requests.delete("http://iceberg.io/v1/namespaces/ecommerce/tables/customers")
response

<Response [200]>

In [20]:
response.content

b''

In [22]:
response = requests.get("http://iceberg.io/v1/namespaces")
response.json()

{'namespaces': []}

In [23]:
response = requests.delete("http://iceberg.io/v1/namespaces/ecommerce/tables/customers")
response

<Response [404]>

In [31]:
response.status_code

404

In [33]:
response.json()["error"]

{'message': 'Table does not exist: ecommerce.customers',
 'type': 'NoSuchTableException',
 'code': 404,
 'stack': ['org.apache.iceberg.exceptions.NoSuchTableException: Table does not exist: ecommerce.customers',
  '\tat org.apache.iceberg.rest.CatalogHandlers.dropTable(CatalogHandlers.java:257)',
  '\tat org.apache.iceberg.rest.RESTCatalogAdapter.handleRequest(RESTCatalogAdapter.java:364)',
  '\tat org.apache.iceberg.rest.RESTServerCatalogAdapter.handleRequest(RESTServerCatalogAdapter.java:42)',
  '\tat org.apache.iceberg.rest.RESTCatalogAdapter.execute(RESTCatalogAdapter.java:527)',
  '\tat org.apache.iceberg.rest.RESTCatalogServlet.execute(RESTCatalogServlet.java:100)',
  '\tat org.apache.iceberg.rest.RESTCatalogServlet.doDelete(RESTCatalogServlet.java:84)',
  '\tat javax.servlet.http.HttpServlet.service(HttpServlet.java:713)',
  '\tat javax.servlet.http.HttpServlet.service(HttpServlet.java:790)',
  '\tat org.eclipse.jetty.servlet.ServletHolder.handle(ServletHolder.java:799)',
  '\ta